In [22]:
import pandas as pd
from flask import Flask, request, jsonify
import json
from elasticsearch import Elasticsearch
import numpy as np
from flask_cors import CORS

In [25]:
df = pd.read_parquet('recipes_parq.parquet')

In [26]:
json.loads(df.loc[df['RecipeId'] == 39 ].iloc[0].to_json())

{'RecipeId': 39.0,
 'Name': 'Biryani',
 'AuthorId': 1567,
 'AuthorName': 'elly9812',
 'CookTime': 'PT25M',
 'PrepTime': 'PT4H',
 'TotalTime': 'PT4H25M',
 'DatePublished': 935932320000,
 'Description': 'Make and share this Biryani recipe from Food.com.',
 'Images': ['https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/picM9Mhnw.jpg',
  'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/39/picHv4Ocr.jpg'],
 'RecipeCategory': 'Chicken Breast',
 'Keywords': ['Chicken Thigh & Leg',
  'Chicken',
  'Poultry',
  'Meat',
  'Asian',
  'Indian',
  'Weeknight',
  'Stove Top'],
 'RecipeIngredientQuantities': ['1',
  '4',
  '2',
  '2',
  '8',
  '1⁄4',
  '8',
  '1⁄2',
  '1',
  '1',
  '1⁄4',
  '1⁄4',
  '1⁄2',
  '1⁄4',
  '2',
  '3',
  None,
  '2',
  '1',
  '1',
  '8',
  '2',
  '1⁄3',
  '1⁄3',
  '1⁄3',
  '6'],
 'RecipeIngredientParts': ['saffron',
  'milk',
  'hot green chili peppers',
  'onions',
  'garlic',
  'clo

In [27]:
df.loc[df['RecipeId'] == 38 ].to_dict('records')[0]

{'RecipeId': 38.0,
 'Name': 'Low-Fat Berry Blue Frozen Dessert',
 'AuthorId': 1533,
 'AuthorName': 'Dancer',
 'CookTime': 'PT24H',
 'PrepTime': 'PT45M',
 'TotalTime': 'PT24H45M',
 'DatePublished': Timestamp('1999-08-09 21:46:00+0000', tz='UTC'),
 'Description': 'Make and share this Low-Fat Berry Blue Frozen Dessert recipe from Food.com.',
 'Images': array(['https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/YUeirxMLQaeE1h3v3qnM_229%20berry%20blue%20frzn%20dess.jpg',
        'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/AFPDDHATWzQ0b1CDpDAT_255%20berry%20blue%20frzn%20dess.jpg',
        'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/UYgf9nwMT2SGGJCuzILO_228%20berry%20blue%20frzn%20dess.jpg',
        'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/PeBMJN2TGSaYks2759BA_20140722_202142.jpg',
  

In [29]:
app = Flask(__name__)
CORS(app)
app.es_client = Elasticsearch("https://localhost:9200", basic_auth=("elastic", "2C2POmj1a75XqoUOs1-h"), ca_certs="~/http_ca.crt")
app.df = pd.read_parquet('recipes_parq.parquet')

# @app.route('/recipes', methods=['GET'])
# def get_recipes():
#     return jsonify(app.recipe_df.to_dict(orient='records'))

@app.route('/recipes', methods=['GET'])
def get_recipe():
    response_object = {'status':200}
    rid = int(request.args.get('rid'))
    recipe = json.loads(app.df.loc[app.df['RecipeId'] == rid ].iloc[0].to_json())
    response_object['recipe'] = recipe
    return response_object

@app.route('/recipes', methods=['POST'])
def get_recipes():
    response_object = {'status':200}
    recipe_id_list = request.get_json()
    recipe_list = json.loads(app.df.loc[recipe_id_list].to_json(orient='records'))
    response_object['list'] = recipe_list
    return response_object

# @app.route('/recipes/<int:id>', methods=['GET'])
# def get_recipe_by_id(id):
#     recipe = app.recipe_df.loc[id]
#     if recipe is None:
#         return jsonify({"error": "Recipe not found"}), 404
#     return jsonify(recipe.to_dict())

@app.route('/search_es', methods=['GET'])
def search_recipes():
    response_object = {'status': "success"}
    query_term = request.args.get('query')
    results = app.es_client.search(index='foodir', query={
            "script_score":{
                "query":{
                    "dis_max": {
                        "queries": [
                            { "match": { "Name": query_term }},
                            { "match": { "RecipeIngredientParts": query_term }},
                            { "match": { "RecipeInstructions": query_term }},
                        ],
                        "tie_breaker": 0.3
                    },
                },
                "script":{
                    "source": "_score * (((doc['ReviewCount'].value * doc['AggregatedRating'].value) + (3 * 32)) / (doc['ReviewCount'].value + 32 ))"
                }
            }
        }, suggest_field='Name', suggest_text=query_term, suggest_mode='missing', size=30)
    
    total_hit = results['hits']['total']['value']
    results_df = pd.DataFrame([[hit['_source'][key] for key in hit['_source']] for hit in results['hits']['hits']], columns=list(app.df.columns))
    results_df['_score'] = [hit['_score'] for hit in results['hits']['hits']]
    response_object['total_hit'] = total_hit
    response_object['results'] = results_df.to_dict('records')
    response_object['suggest'] = results['suggest']
    return response_object

app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Mar/2024 00:46:13] "GET /search_es?query=mushroom HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2024 00:46:14] "GET /search_es?query=mushroom HTTP/1.1" 200 -
